# Backtest (high-level API)

Tutorial for [NautilusTrader](https://nautilustrader.io/docs/) a high-performance algorithmic trading platform and event driven backtester.

[View source on GitHub](https://github.com/nautechsystems/nautilus_trader/blob/develop/docs/getting_started/backtest_high_level.ipynb).

## Overview

This tutorial walks through how to use a `BacktestNode` to backtest a simple EMA cross strategy
on a simulated FX ECN venue using historical quote tick data.

The following points will be covered:
- How to load raw data (external to Nautilus) into the data catalog
- How to set up configuration objects for a `BacktestNode`
- How to run backtests with a  `BacktestNode`

## Prerequisites
- Python 3.11+ installed
- [JupyterLab](https://jupyter.org/) or similar installed (`pip install -U jupyterlab`)
- [NautilusTrader](https://pypi.org/project/nautilus_trader/) latest release installed (`pip install -U nautilus_trader`)

## Imports

We'll start with all of our imports for the remainder of this tutorial.

In [1]:
import shutil
from decimal import Decimal
from pathlib import Path

import pandas as pd

from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.model.data import QuoteTick
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.wranglers import QuoteTickDataWrangler
from nautilus_trader.test_kit.providers import CSVTickDataLoader
from nautilus_trader.test_kit.providers import TestInstrumentProvider

As a once off before we start the notebook - we need to download some sample data for backtesting.

For this example we will use FX data from `histdata.com`. Simply go to https://www.histdata.com/download-free-forex-historical-data/?/ascii/tick-data-quotes/ and select an FX pair, then select one or more months of data to download.

Once you have downloaded the data, set the variable `DATA_DIR` below to the directory containing the data. By default, it will use the users `Downloads/Data/` directory.

In [2]:
DATA_DIR = "~/Downloads/"

In [3]:
path = Path(DATA_DIR).expanduser() / "HISTDATA"
raw_files = list(path.iterdir())
assert raw_files, f"Unable to find any histdata files in directory {path}"
raw_files

FileNotFoundError: [Errno 2] No such file or directory: '/home/zachary81/Downloads/HISTDATA'

## Loading data into the Parquet data catalog

The FX data from `histdata` is stored in CSV/text format, with fields `timestamp, bid_price, ask_price`.
Firstly, we need to load this raw data into a `pandas.DataFrame` which has a compatible schema for Nautilus quotes.

Then we can create Nautilus `QuoteTick` objects by processing the DataFrame with a `QuoteTickDataWrangler`.


In [ ]:
# Here we just take the first data file found and load into a pandas DataFrame
df = CSVTickDataLoader.load(raw_files[0], index_col=0, datetime_format="%Y%m%d %H%M%S%f")
df.columns = ["timestamp", "bid_price", "ask_price"]

# Process quotes using a wrangler
EURUSD = TestInstrumentProvider.default_fx_ccy("EUR/USD")
wrangler = QuoteTickDataWrangler(EURUSD)

ticks = wrangler.process(df)

C:\Users\Zack\AppData\Roaming\Python\Python312\site-packages\nautilus_trader\persistence\loaders.py:58: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


See the [Loading data](../concepts/data) guide for further details.

Next, we simply instantiate a `ParquetDataCatalog` (passing in a directory where to store the data, by default we will just use the current directory).
We can then write the instrument and tick data to the catalog, it should only take a couple of minutes to load the data (depending on how many months).

In [ ]:
CATALOG_PATH = Path.cwd() / "catalog"

# Clear if it already exists, then create fresh
if CATALOG_PATH.exists():
    shutil.rmtree(CATALOG_PATH)
CATALOG_PATH.mkdir(parents=True)

# Create a catalog instance
catalog = ParquetDataCatalog(CATALOG_PATH)

# Write instrument to the catalog
catalog.write_data([EURUSD])

# Write ticks to catalog
catalog.write_data(ticks)

## Using the Data Catalog 

Once data has been loaded into the catalog, the `catalog` instance can be used for loading data for backtests, or simply for research purposes. 
It contains various methods to pull data from the catalog, such as `.instruments(...)` and `quote_ticks(...)` (shown below).

In [ ]:
catalog.instruments()

[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

In [ ]:
start = dt_to_unix_nanos(pd.Timestamp("2022-12-03", tz="UTC"))
end =  dt_to_unix_nanos(pd.Timestamp("2020-12-04", tz="UTC"))

catalog.quote_ticks(instrument_ids=[EURUSD.id.value], start=start, end=end)[:10]

instrument = catalog.instruments()[0]

## Add venues

In [ ]:
venue_configs = [
    BacktestVenueConfig(
        name="SIM",
        oms_type="HEDGING",
        account_type="MARGIN",
        base_currency="USD",
        starting_balances=["1_000_000 USD"],
    ),
]

## Add data

In [ ]:
data_configs = [
    BacktestDataConfig(
        catalog_path=str(ParquetDataCatalog.from_env().path),
        data_cls=QuoteTick,
        instrument_id=instrument.id,
        start_time=start,
        end_time=end,
    ),
]

OSError: 'NAUTILUS_PATH' environment variable is not set.

## Add strategies

In [ ]:
strategies = [
    ImportableStrategyConfig(
        strategy_path="nautilus_trader.examples.strategies.ema_cross:EMACross",
        config_path="nautilus_trader.examples.strategies.ema_cross:EMACrossConfig",
        config={
            "instrument_id": instrument.id,
            "bar_type": "EUR/USD.SIM-15-MINUTE-BID-INTERNAL",
            "fast_ema_period": 10,
            "slow_ema_period": 20,
            "trade_size": Decimal(1_000_000),
        },
    ),
]

## Configure backtest

Nautilus uses a `BacktestRunConfig` object, which enables backtest configuration in one place. It is a `Partialable` object (which means it can be configured in stages); the benefits of which are reduced boilerplate code when creating multiple backtest runs (for example when doing some sort of grid search over parameters).

In [ ]:
config = BacktestRunConfig(
    engine=BacktestEngineConfig(strategies=strategies),
    data=data_configs,
    venues=venue_configs,
)

## Run backtest

Now we can run the backtest node, which will simulate trading across the entire data stream.

In [ ]:
node = BacktestNode(configs=[config])

results = node.run()
results